In this notebook, we will compute the stable coefficients of the census knots or prove that there exist no compatible stable coefficients (and compute N if they exist) and thus obstruct some knots in the SnapPy census from having alternating surgeries.

In [1]:
import snappy
import time
import csv

def torsion_coef(knot):
    '''
    A function that takes the name of a knot and return the torsion coefficients of its normalized Alexander polynomial.
    '''
    M=snappy.Manifold(knot)
    a=M.alexander_polynomial().list()
    t=[]
    for i in range(0,len(a)):
        ti=0
        for j in range(0,len(a)-i):
            ti=ti+j*a[j+i]
        t.append(ti)
        # Next we need to adjust for the correct normalization of the Alexander polynomial
    g=int((len(t)-1)/2)
    r=t[g:2*g]
    T=[]
    Ti=0
    for i in range(1,r[0]+1):
        Ti=r.count(i)+Ti
        T.append(Ti)
    return T

## This code calculates the stable coefficients given the torsion
## coefficients. If no stable coefficients can exist it will return False.
## It takes as input the list [T_1, \dots, T_{t0}]. Where T_k is defined
## as the number if 0\leq i <g such that t_i\leq k.

def Generate_S(n):
    ##Generate the tuples S_k for k=0 to n.
    S=[]
    for k in range(0,n+1):
        S+=[[]]
        j=1
        while j*(j+1)<2*k:
            for t in S[k-(j*(j+1))//2]:
                if j>=t[0]:
                    S[k]+=[[j]+t]
            j+=1

        if j*(j+1)==2*k:
            S[k]+=[[j]]

    return S

def max_pairing(sigma, Sk):
    ##compute max \sigma \cdot \alpha for all \alpha in Sk
    prodlist = [sum([z[0]*z[1] for z in zip(alpha, sigma)]) for alpha in Sk]
    return max(prodlist)

def Calc_Stable(Tlist):
    ##Take the tuple of T_i and determine whether there exists a
    ##tuple of stable coefficients consistent with this.
    t0=len(Tlist)
    g=Tlist[-1]

    ##For indexing reasons append a 0 to the start.
    T=[0]+Tlist
    rho=[]
 
    S=Generate_S(t0)
    k=1
    ##Calculate the \rho_i greater than 2.
    while k<=t0 and T[k]-T[k-1]>2:
        if max_pairing(rho,S[k])<T[k]:
            rho+=[T[k]-T[k-1]]
        ##Check rho decreasing
        if k>1 and rho[-1]>rho[-2]:
            return False
        k+=1

    ##Fill out rho with 2s.
    gsum=sum([(x*(x-1))//2 for x in rho])

    if gsum>g:
        return False
    rho+=[2]*(g-gsum)
    
    ##Finally check that rho is compatible with the T we started with.
    ##pad out rho with some entries equal to one and compute the
    ##relevant maxima.
    sigma=rho+[1]*t0
    for k in range(1,t0):
        if T[k]!=max_pairing(sigma,S[k]):
            return False
    if max_pairing(sigma,S[t0])<g:
        return False
    
    return rho
                 
##print(Calc_Stable([3,5]))
##
##m=[11, 16, 22, 27, 32, 37, 38, 43, 48, 49, 53, 54, 59, 60, 64, 65, 69, 70, 74, 75, 76, 80, 81, 85]
##print(m)
##print(Calc_Stable(m))
##
##tests=[[1],[2],[3],[4],[2,3],[2,3,4],[2,4,5],[3,6],[3,6,9],[3,6,10],[3,4],[4,5],[4,5,6],[4,7,8,9],[4,7,8,9,10],
##       [4,6,8,10],[4,7,8,9],[4,7,8,10],[3,6,7],[3,6,8],[3,6,8,10],[4,7,9,12],
##       [5,9,12,15,17,20,21,22],[7,12,15,19,21,24,27,28,31,33],
##       [5,8,11,13,15,16],[6,9,12,15,17,19,21], [5,7,10,12,13],
##       [6,9,12,15,17,19],[2,6,7]]
##
##for T in tests:
##    print(T, Calc_Stable(T))

## ##Torsion lists for the unknown L-space knots
##T=[8,11,16,19,21,24,27,29,31,33,34]
##T=[8,13,16,21,24,27,30,32,35,36,38,40,42,43]
##print(Calc_Stable(T))


def compute_N(stable_coefficients):
    '''
    Takes as input the stable coefficients and returns the integer N.
    '''
    N=0
    stable_coefficients=sorted(stable_coefficients)
    for s in stable_coefficients:
        N=N+s*s
    values=[]
    for s in stable_coefficients:
        value=s
        for x in stable_coefficients[0:stable_coefficients.index(s)]:
            value=value-x
        values.append(value)
    return N+max(values)

In [2]:
LSpaceKnots=[]
with open('census_L_space_knots.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        LSpaceKnots.append(row[0])
        
LSpaceKnots=LSpaceKnots[1:]

len(LSpaceKnots)

632

In [3]:
start_time = time.time()
census_knots_without_alternating_surgery=[]
census_knots_with_stable_coefficients=[]

for knot in LSpaceKnots:
    T=torsion_coef(knot)
    stab=Calc_Stable(T)
    if stab==False:
        census_knots_without_alternating_surgery.append(knot)
    else:
        census_knots_with_stable_coefficients.append([knot,stab,compute_N(stab)])    

print('Number of L-space census knots without alternating surgeries:',len(census_knots_without_alternating_surgery))
print('Number of L-space census knots with stable coefficients:',len(census_knots_with_stable_coefficients))
print("--- Time taken: %s seconds ---" % ((time.time() - start_time)))

Number of L-space census knots without alternating surgeries: 124
Number of L-space census knots with stable coefficients: 508
--- Time taken: 2.1052589416503906 seconds ---


We print and save the data:

In [4]:
with open("stable_coefficients.csv", "a") as output: #Opens output file # Use the a parameter to add a row
    csvwriter = csv.writer(output, delimiter = ",")
    csvwriter.writerow(('knot','stable coefficients','N'))

for x in census_knots_with_stable_coefficients:
    with open("stable_coefficients.csv", "a") as output: #Opens output file # Use the a parameter to add a row
        csvwriter = csv.writer(output, delimiter = ",")
        csvwriter.writerow((x[0],x[1],x[2]))

In [5]:
census_knots_with_stable_coefficients

[['m016', [3, 2, 2], 19],
 ['m071', [5, 2], 32],
 ['m082', [3, 3, 2, 2], 28],
 ['m103', [5, 3, 2, 2], 44],
 ['m118', [4, 3, 2], 31],
 ['m144', [3, 3, 3, 2, 2], 37],
 ['m194', [5, 3], 37],
 ['m198', [5, 2, 2, 2], 39],
 ['m223', [3, 2, 2, 2], 23],
 ['m239', [4, 3, 2, 2], 35],
 ['m240', [4, 3, 3], 37],
 ['m270', [5, 3, 3], 46],
 ['m276', [5, 4, 2, 2], 51],
 ['m281', [4, 4, 3, 2], 47],
 ['o9_00133', [5, 5, 5, 5, 5, 2], 132],
 ['o9_00168', [5, 5, 5, 5, 5, 3, 2, 2], 144],
 ['o9_00644', [3, 3, 3, 3, 3, 3, 3, 2, 2], 73],
 ['o9_00797', [7, 7, 7, 7, 3, 2, 2], 215],
 ['o9_00815', [7, 7, 7, 7, 4, 3, 2], 227],
 ['o9_01079', [6, 6, 6, 2], 116],
 ['o9_01436', [7, 7, 7, 2, 2], 158],
 ['o9_01496', [7, 7, 7, 5, 2, 2, 2], 186],
 ['o9_01584', [8, 8, 8, 3, 3, 2, 2], 220],
 ['o9_01621', [8, 8, 8, 5, 3], 229],
 ['o9_01680', [8, 8, 8, 3, 2, 2], 211],
 ['o9_01765', [8, 8, 8, 5, 3, 3], 238],
 ['o9_01936', [7, 2], 58],
 ['o9_01953', [5, 5, 5, 5, 5, 3], 137],
 ['o9_01955', [5, 5, 5, 5, 5, 2, 2, 2], 139],
 ['o9_02